# 4. Database

MySQL is a relational database management system (RDBMS) developed by Oracle that is based on structured query language (SQL). A database is a structured collection of data.

See this [tutorial](https://www.w3schools.com/MySQL/default.asp) for more information.

![screen](../img/framework_err.png)

Note: To continue with this tutorial, you should already install and setup the database. If you haven't done it, please see the instruction at the root of this project.

## Setup

We need to install the `mysql-connector-python` package to access the database through python. For detail documentation you can see [here](https://dev.mysql.com/doc/connector-python/en/). 

```terminal
pip install mysql-connector-python
```

Once installed, you can import the package with the command `import mysql.connector`

In [1]:
import mysql.connector

Now, we need to setup the connection config to our database.


In [2]:
# MySQL connection parameters
mysql_config = {
    'user': 'user_1',
    'password': '1234',
    'host': 'localhost',
    'database': 'framework'
}

Then, we can open the connection to the database. 

```python
conn = mysql.connector.connect(**mysql_config) # Opening the connection
cursor = conn.cursor()

...
# Do your database transaction here 
... 

cursor.close() # close the cursor object
conn.close() # close the connection object
```

At the end, we need to close the connection of the cursor and the conn object.




## Playing with the database

Now, you are ready to play with the database where you can query, add, update, and delete the data.

Note: If you are unfamiliar with SQL language, please see this [W3School's tutorial](https://www.w3schools.com/MySQL/default.asp)

### Getting the data from the database

For example, we want to query the list of active hardware exists for our platform with IBM as the provider.

In [24]:
# Connect to the MySQL database
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

sql = """
SELECT hw_name, number_of_qubit, 2q_native_gates FROM hardware WHERE hw_provider = %s and status != 'deactivated';
"""

parameters = ("IBM",) # parameters file should be a list type

cursor.execute(sql, parameters)

results = cursor.fetchall()

for res in results:
    print(res)

cursor.close()
conn.close()

('ibmq_kolkata', 27, 'cx')
('ibmq_mumbai', 27, 'cx')
('ibm_algiers', 27, 'cx')
('ibm_brisbane', 127, 'ecr')
('ibm_cairo', 27, 'cx')
('ibm_cusco', 127, 'ecr')
('ibm_hanoi', 27, 'cx')
('ibm_kawasaki', 127, 'ecr')
('ibm_kyiv', 127, 'cx')
('ibm_kyoto', 127, 'ecr')
('ibm_nazca', 127, 'ecr')
('ibm_osaka', 127, 'ecr')
('ibm_peekskill', 27, 'cx')
('ibm_quebec', 127, 'ecr')
('ibm_sherbrooke', 127, 'ecr')


## Implementation in our Platform

The database is used in many aspects for our platform.
- save the hardware information where we used them for our noise-aware compilation technique
- save the circuits have been used in our experiment
- save the compilation techniques used in our experiment
- save the results of our experiments (quantum result, classical result and evaluation metrics)

### Getting the hardware information (calibration data) from IBM 

This command is to retrieve calibration data from IBM. For example, here is the JSON file generated for `ibm_sherbrooke`. The JSON file contains all the information related to the hardware.


In [14]:
import requests
json_url = "https://api-qcon.quantum-computing.ibm.com/api/Backends/ibm_sherbrooke/properties"
response = requests.get(json_url)
print(response.json()["backend_name"])

ibm_sherbrooke


For more detail, you can see the [python script](retrieve_calibration_data.py) where it retrieve all the hardware information that are listed in the hardware table.

These are the data that you will get after processing and put them into data frame

In [30]:
import pandas as pd

# Connect to the MySQL database
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

sql = """
SELECT calibration_id, qubit, id_error, reset_error, sx_error, x_error 
FROM ibm_one_qubit_gate_spec WHERE calibration_id = %s;
"""

 # insert to circuit
cursor.execute(sql, (36535,))

results = cursor.fetchall()

calibration_id = []
qubit = []
id_error = []
reset_error = []
sx_error = []
x_error = []


for res in results:
    _calibration_id, _qubit, _id_error , _reset_error, _sx_error, _x_error = res

    calibration_id.append(_calibration_id)
    qubit.append(_qubit)
    id_error.append(float(_id_error))
    reset_error.append(float(_reset_error))
    sx_error.append(float(_sx_error))
    x_error.append(float(_x_error))

cursor.close()
conn.close()

df = pd.DataFrame({
    'calibration_id': calibration_id,
    'qubit': qubit,
    'id_error': id_error,
    'reset_error': reset_error,
    'sx_error': sx_error,
    'x_error': x_error
    })

In [33]:
df

,calibration_id,qubit,id_error,reset_error,sx_error,x_error
0,36535,0,0.000145,0.0,0.000145,0.000145
1,36535,1,0.000126,0.0,0.000126,0.000126
2,36535,2,0.000148,0.0,0.000148,0.000148
3,36535,3,0.000472,0.0,0.000472,0.000472
4,36535,4,0.000834,0.0,0.000834,0.000834
...,...,...,...,...,...,...
122,36535,122,0.000264,0.0,0.000264,0.000264
123,36535,123,0.000418,0.0,0.000418,0.000418
124,36535,124,0.001674,0.0,0.001674,0.001674
125,36535,125,0.001138,0.0,0.001138,0.001138
